Basándonos en el ejemplo de validación cruzada empleando Pipelines perteneciente a la sección 4.4.5 de la unidad didáctica:

In [1]:
# Cargo los datos
games_df = spark.read.csv('../data/games.csv', header=True, sep=',', inferSchema=True)
games_red_df = games_df.selectExpr('winner',
                                   'gameDuration as duration',
                                   'firstBlood',
                                   'firstTower',
                                   'firstInhibitor',
                                   'firstBaron',
                                   'firstDragon',
                                   'firstRiftHerald',
                                   't1_champ1id',
                                   't1_champ2id',
                                   't1_champ3id',
                                   't1_champ4id',
                                   't1_champ5id',
                                   't1_towerKills',
                                   't1_inhibitorKills',
                                   't1_baronKills',
                                   't1_dragonKills',
                                   't1_riftHeraldKills',
                                   't2_champ1id',
                                   't2_champ2id',
                                   't2_champ3id',
                                   't2_champ4id',
                                   't2_champ5id',
                                   't2_towerKills',
                                   't2_inhibitorKills',
                                   't2_baronKills',
                                   't2_dragonKills',
                                   't2_riftHeraldKills')
## Reducción de datos para acelerar los procesos. //TODO: Desactivar esto en la ejecución final
#print("Antes: games_df:", games_df.count(), "games_red_df:", games_red_df.count())
#games_df = games_df.sample(fraction = 0.1)
#games_red_df = games_red_df.sample(fraction = 0.1)
#print("Despues: games_df:", games_df.count(), "games_red_df:", games_red_df.count())

1. (1 punto) Emplea validación cruzada con 5 bloques. Añade en este y en los siguientes bloques de entrenamiento, el tiempo que dura cada ejecución de entrenamiento, para ello ayúdate del método now de la librería datetime de Python.

In [2]:
from datetime import datetime as dt
from pyspark.ml.feature import CountVectorizer, Binarizer, StandardScaler, VectorAssembler, OneHotEncoderEstimator, PCA
from pyspark.sql.types import DoubleType
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import expr

t_inicial_logistica = dt.now()

[train_df, test_df]=games_red_df.randomSplit([0.7, 0.3]) 
train_df = train_df.withColumn('winner', train_df.winner.cast(DoubleType())) 
test_df = test_df.withColumn('winner', test_df.winner.cast(DoubleType())) 
new_column_expression = expr('split( concat_ws( "," ,t1_champ1id, t1_champ2id, t1_champ3id, t1_champ4id, t1_champ5id), "," )' ) 
train_df = train_df.withColumn('t1_members_str',new_column_expression) 
test_df = test_df.withColumn('t1_members_str', new_column_expression) 
new_column_expression = expr('split( concat_ws( "," ,t2_champ1id, t2_champ2id, t2_champ3id, t2_champ4id, t2_champ5id), "," )' ) 
train_df = train_df.withColumn('t2_members_str',new_column_expression) 
test_df = test_df.withColumn('t2_members_str', new_column_expression) 
cv1 = CountVectorizer(inputCol='t1_members_str', outputCol='t1_members') 
cv2 = CountVectorizer(inputCol='t2_members_str', outputCol='t2_members') 
binarizer=Binarizer(inputCol='winner', outputCol='winner_b', threshold=1) 
columns = ['firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald'] 
new_columns = [ 'b_firstBlood', 'b_firstTower', 'b_firstInhibitor', 'b_firstBaron', 'b_firstDragon', 'b_firstRiftHerald' ] 
ohe = OneHotEncoderEstimator(inputCols=columns, outputCols=new_columns,dropLast=False) 
columns = ["duration", "t1_towerKills", "t1_inhibitorKills", "t1_baronKills", "t1_dragonKills", "t1_riftHeraldKills", "t2_towerKills", "t2_inhibitorKills", "t2_baronKills", "t2_dragonKills", "t2_riftHeraldKills"] 
assembler1 = VectorAssembler(inputCols=columns, outputCol="assembledColumns")
scaler = StandardScaler(inputCol="assembledColumns", outputCol="standardColumns", withStd=True, withMean=True)
columns = ['t1_members', 't2_members', 'b_firstBlood', 'b_firstBaron', 'b_firstDragon', 'b_firstInhibitor', 'b_firstRiftHerald', 'b_firstTower', 'standardColumns'] 
assembler2 = VectorAssembler(inputCols=columns, outputCol='features') 
pca = PCA(inputCol='features', outputCol='red_features') 
logistic = LogisticRegression(labelCol='winner_b',featuresCol='red_features')

from pyspark.ml.tuning import ParamGridBuilder 
params_logistica = ParamGridBuilder().addGrid(logistic.elasticNetParam, [0, 0.33]).\
addGrid(pca.k, [50, 100]).addGrid(logistic.regParam, [0.1, 0.33]).addGrid(logistic.maxIter,[50]).build()


from pyspark.ml.tuning import CrossValidator 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', predictionCol='prediction', labelCol='winner_b')
pipeline = Pipeline().setStages([cv1, cv2, binarizer, ohe, assembler1, scaler, assembler2, pca, logistic]) 
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(params_logistica).setNumFolds(5)
model_logistica = cv.fit(train_df)

t_final_logistica = dt.now()
t_ejecucion_logistica = t_final_logistica - t_inicial_logistica
print(f"Tiempo de ejecución (hh:mm:ss.ms): {t_ejecucion_logistica}")
for (result, config) in zip(model_logistica.avgMetrics, params_logistica) :
    print(result, config[pca.k], config[logistic.regParam], config[logistic.elasticNetParam])

ev_logistica = evaluator.evaluate(model_logistica.transform(test_df))
ev_logistica

Tiempo de ejecución (hh:mm:ss.ms): 0:02:26.847367
0.9603631507638754 50 0.1 0.0
0.9596430077838859 50 0.33 0.0
0.9604198923050231 100 0.1 0.0
0.959419986743417 100 0.33 0.0
0.9471384860946084 50 0.1 0.33
0.9271863375519438 50 0.33 0.33
0.9471384860946084 100 0.1 0.33
0.9271863375519438 100 0.33 0.33


0.9617070654976793

2. (2 puntos) Amplía la horquilla de valores probados para la regresión Logística:

    a. elasticNetParam, probando 2 valores más entre 0 y 1.

    b. regParam, probando 3 valores más entre 0 y 10.

    c. maxIter, probando 2 valores más entre 1 y 100.

In [3]:
t_inicial_logistica2 = dt.now()
params_logistica2 = ParamGridBuilder().addGrid(logistic.elasticNetParam, [0, 0.33, 0.5, 0.75]).\
addGrid(pca.k, [50, 100]).addGrid(logistic.regParam, [0.1, 0.33, 3, 5]).addGrid(logistic.maxIter,[20,50,80]).build()
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(params_logistica2).setNumFolds(5)
model_logistica2 = cv.fit(train_df)

t_final_logistica2 = dt.now()
t_ejecucion_logistica2 = t_final_logistica2 - t_inicial_logistica2
print(f"Tiempo de ejecución (hh:mm:ss.ms): {t_ejecucion_logistica2}")
for (result, config) in zip(model_logistica2.avgMetrics, params_logistica2) :
    print(result, config[pca.k], config[logistic.regParam], config[logistic.elasticNetParam], config[logistic.maxIter])

ev_logistica2 = evaluator.evaluate(model_logistica2.transform(test_df))
ev_logistica2

Tiempo de ejecución (hh:mm:ss.ms): 0:21:42.198405
0.9603631507638754 50 0.1 0.0 20
0.9603631507638754 50 0.1 0.0 50
0.9603631507638754 50 0.1 0.0 80
0.9596430077838859 50 0.33 0.0 20
0.9596430077838859 50 0.33 0.0 50
0.9596430077838859 50 0.33 0.0 80
0.948004990199687 50 3.0 0.0 20
0.948004990199687 50 3.0 0.0 50
0.948004990199687 50 3.0 0.0 80
0.9280463297632416 50 5.0 0.0 20
0.9280463297632416 50 5.0 0.0 50
0.9280463297632416 50 5.0 0.0 80
0.9604198923050231 100 0.1 0.0 20
0.9604198923050231 100 0.1 0.0 50
0.9604198923050231 100 0.1 0.0 80
0.959419986743417 100 0.33 0.0 20
0.959419986743417 100 0.33 0.0 50
0.959419986743417 100 0.33 0.0 80
0.9487026765292466 100 3.0 0.0 20
0.9487026765292466 100 3.0 0.0 50
0.9487026765292466 100 3.0 0.0 80
0.9301606807772531 100 5.0 0.0 20
0.9301606807772531 100 5.0 0.0 50
0.9301606807772531 100 5.0 0.0 80
0.9471384860946084 50 0.1 0.33 20
0.9471384860946084 50 0.1 0.33 50
0.9471384860946084 50 0.1 0.33 80
0.9271863375519438 50 0.33 0.33 20
0.9271863

0.9617070654976793

3. (2.5 puntos) Realiza también un experimento con el mismo conjunto de datos, pero esta vez empleando un Random Forest
(https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#random-forest-classifier, https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.tree.RandomForestModel) como modelo a entrenar. Para este modelo probaremos las siguientes horquillas de parámetros:

    a. numTrees, probando 3 valores entre 1 y 100.
    b. maxDepth, probando 3 valores entre 1 y 10.

In [4]:
t_inicial_rfc = dt.now()
from pyspark.ml.classification import RandomForestClassifier
ramdomForest = RandomForestClassifier(labelCol='winner_b',featuresCol='red_features')

params_rfc = ParamGridBuilder().addGrid(ramdomForest.maxDepth, [2, 5, 9]).addGrid(pca.k, [50, 100]).addGrid(ramdomForest.numTrees, [10, 50, 100]).build()
pipeline = Pipeline().setStages([cv1, cv2, binarizer, ohe, assembler1, scaler, assembler2, pca, ramdomForest]) 
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', predictionCol='prediction', labelCol='winner_b')
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(params_rfc).setNumFolds(5)
model_rfc = cv.fit(train_df)

t_final_rfc = dt.now()
t_ejecucion_rfc = t_final_rfc - t_inicial_rfc
print(f"Tiempo de ejecución (hh:mm:ss.ms): {t_ejecucion_rfc}")
for (result, config) in zip(model_rfc.avgMetrics, params_rfc) :
    print(result, config[pca.k], config[ramdomForest.numTrees], config[ramdomForest.maxDepth])

ev_rfc = evaluator.evaluate(model_rfc.transform(test_df))
ev_rfc

Tiempo de ejecución (hh:mm:ss.ms): 0:09:14.806680
0.9282494687547282 50 10 2
0.9303454709719057 50 50 2
0.930875238895212 50 100 2
0.926577295885161 100 10 2
0.9317668014799373 100 50 2
0.9304626829208582 100 100 2
0.9373904520924319 50 10 5
0.9380006946607123 50 50 5
0.9376964903878597 50 100 5
0.9378096745968093 100 10 5
0.9361189228745124 100 50 5
0.9368649004331362 100 100 5
0.9471446523426482 50 10 9
0.9487313043634578 50 50 9
0.950034172164556 50 100 9
0.9444828723759648 100 10 9
0.9450860572112703 100 50 9
0.9458679025080009 100 100 9


0.9513280041258381

4. (3 puntos) Prueba ahora sobre el mismo conjunto de datos, pero esta vez utilizando un modelo Gradient-boosted tree (https://spark.apache.org/docs/2.2.0/mlclassificationregression.html#gradient-boosted-tree-classifier, https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.tree.GradientBoostedTreesModel) como algoritmo a entrenar y eliminando la reducción de variables de PCA. Para este modelo probaremos las siguientes horquillas de parámetros:

    a. maxIter, probando 2 valores entre 10 y 100.
    
    b. maxDepth, probando 2 valores entre 1 y 10.

In [5]:
t_inicial_gbt = dt.now()

from pyspark.ml.classification import GBTClassifier
GradientBoostedTree = GBTClassifier(labelCol='winner_b',featuresCol='features')

params_gbt = ParamGridBuilder().addGrid(GradientBoostedTree.maxIter, [30, 75]).addGrid(GradientBoostedTree.maxDepth, [2, 8]).build()
pipeline = Pipeline().setStages([cv1, cv2, binarizer, ohe, assembler1, scaler, assembler2, GradientBoostedTree]) 
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', predictionCol='prediction', labelCol='winner_b')
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(params_gbt).setNumFolds(5)
model_gbt = cv.fit(train_df)

t_final_gbt = dt.now()
t_ejecucion_gbt = t_final_gbt - t_inicial_gbt
print(f"Tiempo de ejecución (hh:mm:ss.ms): {t_ejecucion_gbt}")
for (result, config) in zip(model_gbt.avgMetrics, params_gbt) :
    print(result, config[GradientBoostedTree.maxIter], config[GradientBoostedTree.maxDepth])

ev_gbt = evaluator.evaluate(model_gbt.transform(test_df))
ev_gbt

Tiempo de ejecución (hh:mm:ss.ms): 0:22:02.053396
0.9546631187069472 30 2
0.9668629164122036 30 8
0.9615503589697858 75 2
0.9671941207402944 75 8


0.9663486333161423

(1.5 punto) Compara los resultados obtenidos para los experimentos que hayas realizado. ¿Qué configuración ha sido la más exitosa? Calcula el tiempo medio por número de modelos entrenados respecto del tiempo total, ¿cuál ha tardado más? ¿Merece la mejora en los resultados frente al tiempo empleado? Genera una celda de texto incluyendo un párrafo con una conclusión.

In [6]:
modelos = ["logistica", "logistica2", "rfc", "gbt"]
for mo in modelos:
    n_model = "model_"+mo # Nombre del modelo
    #print (n_model)
    modelo = eval(n_model)
    ev_model = "ev_"+mo
    evaluador = eval(ev_model)
    #print(evaluador)
    params_model = "params_"+mo
    parametros = eval(params_model)
    metricas_model = modelo.avgMetrics
    cantidad_metricas_model = len(metricas_model)
    #print(metricas_model)
    #print(cantidad_metricas_model)
    t_inicial = "t_inicial_"+mo
    t_inicial_model = eval(t_inicial)
    t_final = "t_final_"+mo
    t_final_model = eval(t_final)
    t_diferencia = t_final_model - t_inicial_model
    tiempo_por_modelo = t_diferencia / cantidad_metricas_model
    mejor_modelo = max(metricas_model)
    mejores_parametros = ""
    for (result, config) in zip(metricas_model, parametros) :
        if(result == mejor_modelo):
            mejores_parametros = config
    

    print("El modelo '", n_model, "' tiene una precisión del ",evaluador,"%.",\
          "\nSe han entrenado ", cantidad_metricas_model, " modelos."\
          "\nHa tardado (hh:mm:ss.ms)",t_diferencia,\
          "\nEl tiempo medio por modelo es ", tiempo_por_modelo,\
          "\nEl mejor modelo se ha ajustado un ",mejor_modelo, "%",\
          sep='')
    print("Los mejores parámetros son:")
    for (key,value) in zip(config.keys(), config.values()):
        print(key.name,": ", value, sep="")
    print("\n")
    
    

El modelo 'model_logistica' tiene una precisión del 0.9617070654976793%.
Se han entrenado 8 modelos.
Ha tardado (hh:mm:ss.ms)0:02:26.847367
El tiempo medio por modelo es 0:00:18.355921
El mejor modelo se ha ajustado un 0.9604198923050231%
Los mejores parámetros son:
elasticNetParam: 0.33
k: 100
regParam: 0.33
maxIter: 50


El modelo 'model_logistica2' tiene una precisión del 0.9617070654976793%.
Se han entrenado 96 modelos.
Ha tardado (hh:mm:ss.ms)0:21:42.198405
El tiempo medio por modelo es 0:00:13.564567
El mejor modelo se ha ajustado un 0.9604198923050231%
Los mejores parámetros son:
elasticNetParam: 0.75
k: 100
regParam: 5.0
maxIter: 80


El modelo 'model_rfc' tiene una precisión del 0.9513280041258381%.
Se han entrenado 18 modelos.
Ha tardado (hh:mm:ss.ms)0:09:14.806680
El tiempo medio por modelo es 0:00:30.822593
El mejor modelo se ha ajustado un 0.950034172164556%
Los mejores parámetros son:
maxDepth: 9
k: 100
numTrees: 100


El modelo 'model_gbt' tiene una precisión del 0.96634

La configuración más exitosa ha sido la última; que utiliza un clasificador "Gradient-boosted tree". Ha logrado una precisión del 96,72% en los datos de test. Sin embargo, también ha sido la que más ha tardado. En entrenar 4 modelos ha empleado más de 20 minutos. Esto es, a más de 5:30 minutos por modelo, y la precisión obtenida no difiere mucho de las demás configuraciones. La peor, logra 95% en poco más de 30 segundos por modelo (tercer modelo). La más rápida, que utiliza regresión logística, logra una precisión del 96,04% en los datos de test, empleando menos de 14 segundos por modelo (segundo modelo). Curiosamente, esta es la misma precisión que se logra con esta misma clase de regresión y disminuyendo el número de parámetros a ajustar (primer modelo). El segundo modelo tarda más tiempo porque combinan más parámetros, de forma que el tiempo total es excesivo. Curiosamente, la configuración de parámetros elegida no es la misma que en el primer modelo, pero obtiene la misma precisión con los datos de test.

En conclusión. Teniendo en cuenta los resultados, el mejor modelo es el primero, que utiliza regresión logística y pocos parámetros, porque logra una precisión muy alta, en un tiempo total muy bajo, menor a 2:30 minutos.